In [1]:
from langchain.agents import initialize_agent, Tool
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
from langchain_groq import ChatGroq
load_dotenv()
groq_key = os.getenv("GROQ_API_KEY")
groq_key

'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'

In [12]:
groq_model = ChatGroq(
    model="qwen-qwq-32b",
    groq_api_key=groq_key)

In [5]:
from langchain_community.utilities.wikipedia import  WikipediaAPIWrapper
search_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to search Wikipedia for information on any topic."
)

In [6]:
tool = [search_tool]
agent = initialize_agent(
    tool,
    groq_model,
    agent="zero-shot-react-description",
    verbose=True
)

C:\Users\Bichu George\AppData\Local\Temp\ipykernel_21056\2661941660.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [13]:
response = agent.run("Who is Elon Musk?")



> Entering new AgentExecutor chain...
I should gather information about who Elon Musk is, focusing on his significant roles and contributions.
Action: Wikipedia Search
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman known for his leadership of Tesla, SpaceX, and X (formerly Twitter). Since 2025, he has been a senior advisor to United States President Donald Trump and the de facto head of the Department of Government Efficiency (DOGE). Musk is the wealthiest person in the world; as of March 2025, Forbes estimates his net worth to be US$345 billion. He was named Time magazine's Person of the Year in 2021.
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada. He graduated from the University of Pennsylvania in the U.S. before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an on

### SQLite Database

In [7]:
import sqlite3
connection = sqlite3.connect(
    "example.db"
)
cursor = connection.cursor()
# Create a table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS inventory (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_name TEXT NOT NULL,
        stock_quantity INTEGER NOT NULL
    )
''')

cursor.executemany(
    """
    INSERT INTO inventory (product_name, stock_quantity)
    VALUES (?, ?)
    """,
    [
        ("Product A", 100),
        ("Product B", 200),
        ("Product C", 300),
        ("Product D", 100),
        ("Product E", 200),
        ("Product F", 300),
    ]
)

In [8]:
connection.commit()
connection.close()

In [9]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit


In [10]:
db = SQLDatabase.from_uri("sqlite:///example.db")

In [13]:
toolkit = SQLDatabaseToolkit(db=db, llm=groq_model)
tools = toolkit.get_tools()
agent = initialize_agent(
    tools,
    groq_model,
    agent="zero-shot-react-description",
    verbose=True
)
response = agent.run("How many units of product A are in stock?")



> Entering new AgentExecutor chain...

<think>
Okay, I need to figure out how many units of product A are in stock. Let's start by understanding what tables and columns I have available. I should first list the tables in the database using sql_db_list_tables.

Action: sql_db_list_tables
Action Input: 

Observation: inventory
Thought:
<think>
Okay, the user asked how many units of product A are in stock. Let me start by figuring out which tables I need to look at. The first step is to check what tables are available. I used sql_db_list_tables and got "inventory" as the only table. So the data must be in the inventory table.

Now I need to know what columns are in the inventory table. Maybe there's a product name or ID and a stock quantity. Let me use sql_db_schema on the inventory table to see its structure.

Action: sql_db_schema
Action Input: inventory

Suppose the schema shows columns like product_id, product_name, units_in_stock, and maybe others. Let's say the relevant columns ar

## Arxiv Query Run

In [14]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun

In [16]:
arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors or keywords."
)

In [17]:
tools = [arxiv_tool]
agent = initialize_agent(
    tools=tools,
    llm=groq_model,
    agent="zero-shot-react-description",
    verbose=True
)

In [25]:
query = "Search for the latest research papers on object detection"
response = agent.run(query)
print(f"Response from Agent: {response}")



> Entering new AgentExecutor chain...

<think>
Okay, the user wants the latest research papers on object detection. I need to use the Arxiv Search tool. Let me think about how to approach this.

First, I should make sure the query is precise. "Object detection" is the main topic, but maybe adding related terms could help. Wait, the tool allows keywords, so maybe just using "object detection" as the query. But I should check if there's a specific time frame they want. The user mentioned "latest," so perhaps the tool defaults to recent papers or I can specify a date range. Wait, does the Arxiv Search tool have parameters for date? The tool's parameters aren't fully detailed here, but maybe the action input can include parameters like the search terms and maybe a filter. Hmm, the tool's description says to provide topics, authors, or keywords, so maybe just the keywords "object detection" along with a note on recency.

Alternatively, maybe the tool automatically prioritizes the most rec